In [ ]:
!pip install easyocr

     |████████████████████████████████| 48.9MB 62kB/s 


In [ ]:
import easyocr

In [ ]:
reader = easyocr.Reader(['en'])

In [ ]:
import matplotlib.pyplot as plt
import cv2
import easyocr
from pylab import rcParams
from IPython.display import Image


In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
import glob
from PIL import Image
files = glob.glob("*.jpg")
print(len(files))
files = sorted(files)

30


In [ ]:
print(files)

['00000.jpg', '00001.jpg', '00002.jpg', '00003.jpg', '00004.jpg', '00005.jpg', '00006.jpg', '00007.jpg', '00008.jpg', '00009.jpg', '00010.jpg', '00011.jpg', '00012.jpg', '00013.jpg', '00014.jpg', '00015.jpg', '00016.jpg', '00017.jpg', '00018.jpg', '00019.jpg', '00020.jpg', '00021.jpg', '00022.jpg', '00023.jpg', '00024.jpg', '00025.jpg', '00026.jpg', '00027.jpg', '00028.jpg', '00029.jpg']


In [ ]:
from IPython.display import Image

output1 = []
for i in range(0, len(files)):
    file_name = files[i]
    print(file_name)
    output = reader.readtext(file_name)
    output1.append(output)

print(len(output1))
#file_name = "00004.jpg"
#image = Image(file_name)

00000.jpg
00001.jpg
00002.jpg
00003.jpg
00004.jpg
00005.jpg
00006.jpg
00007.jpg
00008.jpg
00009.jpg
00010.jpg
00011.jpg
00012.jpg
00013.jpg
00014.jpg
00015.jpg
00016.jpg
00017.jpg
00018.jpg
00019.jpg
00020.jpg
00021.jpg
00022.jpg
00023.jpg
00024.jpg
00025.jpg
00026.jpg
00027.jpg
00028.jpg
00029.jpg
30


In [ ]:
cordin = []
f11 = []
f22 = []

for l in range(0, len(output1)):
    fname = files[l]
    image = cv2.imread(fname)

    f1 = []
    f2 = []

    for j in range(0, len(output1[l])):
        cord = output1[l][j][0]
        x1, y1 = [int(max(idx)) for idx in zip(*cord)]
        x2, y2 = [int(min(idx)) for idx in zip(*cord)]
        a, b, c = image[y1][x1][0], image[y1][x1][1], image[y1][x1][2]
        a1, b1, c1 = image[y2][x2][0], image[y2][x2][1], image[y2][x2][2]
        if (len(f1)==0):
            f1.append([a,b,c,a1,b1,c1])
            f2.append([x2, y2, x1, y1])
            x_max = x1
            y_max = y1
            x_min = x2
            y_min = y2
        else:
            x1, y1 = [int(max(idx)) for idx in zip(*cord)]
            x2, y2 = [int(min(idx)) for idx in zip(*cord)]
            a2, b2, c2 = image[y1][x1][0], image[y1][x1][1], image[y1][x1][2]
            a3, b3, c3 = image[y2][x2][0], image[y2][x2][1], image[y2][x2][2]
            flag = 0
            for i in range(0, len(f1)):
                a, b, c, a1, b1, c1 = f1[i]
                x_min, y_min, x_max, y_max = f2[i]
                if (((abs(int(a2) - int(a))) < 10 and (abs(int(b2) - int(b))) < 10 and (abs(int(c2) - int(c))) < 10) or ((abs(int(a3) - int(a1))) < 10 and (abs(int(b3) - int(b1))) < 10 and (abs(int(c3) - int(c1))) < 10)):
                    flag = 1
                    if (x1 > x_max):
                        x_max = x1
                    if (y1 > y_max):
                        y_max = y1 
                    if (x2 < x_min):
                        x_min = x2
                    if (y2 < y_min):
                        y_min = y2
                    f2[i][0] = x_min
                    f2[i][1] = y_min
                    f2[i][2] = x_max
                    f2[i][3] = y_max
                    break
            if (flag == 0):
                f1.append([a2, b2, c2, a3, b3, c3])
                f2.append([x2, y2, x1, y1])

    f22.append(f2)  


In [ ]:
import numpy as np
y_pred = []
for i in range(0, len(files)):
  if(i!=7 or i!=23):
    im = cv2.imread(files[i], 0)
    a1 = np.zeros((im.shape[0], im.shape[1]))
    for k in range(0, len(f22[i])):
        x_min, y_min, x_max, y_max = f22[i][k]
        for i1 in range(x_min, x_max):
            for j1 in range(y_min, y_max):
              a1[j1][i1] = 255
    y_pred.append(a1)

In [ ]:
import pickle
a11 = pickle.load( open( "text_boxes.pkl", "rb" ) )
print(len(a11[0]))

2


In [ ]:
## For QSD-1 W1

y_truth = []
for i in range(0, len(files)):
  if(i!=7 or i!=23):
    im = cv2.imread(files[i], 0)
    a, b = im.shape
    a1 = np.zeros((im.shape[0], im.shape[1]))
    for k in range(0, len(a11[i])):
        x_min, y_min, x_max, y_max = a11[i][k]
        for i1 in range(int(x_min), x_max):
            for j1 in range(y_min, y_max):
                a1[j1][i1] = 255
    y_truth.append(a1)

In [ ]:
iou = []
for i in range(0, len(y_pred)):
    ax = y_pred[i]
    bx = y_truth[i]
    tp = np.sum(np.logical_and(bx == 255, ax == 255))
    fn = np.sum(np.logical_and(bx == 0, ax == 255))
    fp = np.sum(np.logical_and(bx == 255, ax == 0))
    iou.append([tp/(tp + fn + fp)])
print(np.mean(iou))

0.7278499048375191
